In [2]:
# Importing necessary libraries
import numpy as np  # NumPy library for numerical operations
import pandas as pd  # Pandas library for data manipulation and analysis
from statsmodels.tsa.stattools import adfuller  # Function for Augmented Dickey-Fuller test
import matplotlib.pyplot as plt  # Matplotlib's pyplot for plotting graphs


In [3]:
# Reading and initial processing of data
columns_to_load = ['PlantKey', 'Temperature', 'SunRad', 'Ratio', 'ValueTime']

# Read the Excel file with specified columns
data = pd.read_excel("data/PrelData ExJobb.xlsx", usecols=columns_to_load, index_col= "ValueTime",parse_dates=True)

#data = pd.read_csv("data/PrelData ExJobb(Blad1).csv", usecols=columns_to_load, index_col= "ValueTime",parse_dates=True)
#index_col='ValueTime'
data.dtypes


#Plantkey temp sunrad ratio ValueTime


pd.set_option('display.max_colwidth', None)
data # Displaying the first few rows of the dataset


,PlantKey,Ratio,SunRad,Temperature
ValueTime,,,,
2023-01-01,FBO0023,96.956513,180.0,-0.925000
2023-01-02,FBO0023,0.000000,0.0,2.625000
2023-01-03,FBO0023,87.450395,650.0,-1.441667
2023-01-04,FBO0023,0.000000,0.0,-3.600000
2023-01-05,FBO0023,0.000000,0.0,-2.979167
...,...,...,...,...
2023-12-27,SGA0001,0.434946,720.0,-2.191667
2023-12-28,SGA0001,13.637467,40.0,0.941667
2023-12-29,SGA0001,12.914114,40.0,4.262500


In [4]:
data.dtypes


PlantKey        object
Ratio          float64
SunRad         float64
Temperature    float64
dtype: object

In [5]:
data.dtypes


PlantKey        object
Ratio          float64
SunRad         float64
Temperature    float64
dtype: object

In [6]:
data


,PlantKey,Ratio,SunRad,Temperature
ValueTime,,,,
2023-01-01,FBO0023,96.956513,180.0,-0.925000
2023-01-02,FBO0023,0.000000,0.0,2.625000
2023-01-03,FBO0023,87.450395,650.0,-1.441667
2023-01-04,FBO0023,0.000000,0.0,-3.600000
2023-01-05,FBO0023,0.000000,0.0,-2.979167
...,...,...,...,...
2023-12-27,SGA0001,0.434946,720.0,-2.191667
2023-12-28,SGA0001,13.637467,40.0,0.941667
2023-12-29,SGA0001,12.914114,40.0,4.262500


In [7]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

# Apply the function to your data
#cleaned_data = remove_outliers(data, 'Ratio')


data.describe()  # Describing the statistical aspects of data



def find_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Apply the function to your data column
#outliers = find_outliers_iqr(data, 'Ratio')
#print(f"Number of outliers detected: {len(outliers)}")



def remove_negative_values(data, column):
   
    filtered_data = data[data[column] >= 0]
    return filtered_data


def remove_Positive_values(data, column):
    filtered_data = data[data[column] < 200]
    return filtered_data
data=remove_negative_values(data,"Ratio")
data=remove_Positive_values(data,"Ratio")


data.describe()
#cleaned_data_final.describe()


,Ratio,SunRad,Temperature
count,7451.000000,7451.000000,7451.000000
mean,72.490525,2674.612804,11.364709
std,35.778067,2386.141036,9.799562
min,0.000000,0.000000,-13.329167
25%,53.422054,450.000000,2.847917
50%,80.164578,2000.000000,10.708333
75%,92.982321,4610.000000,19.852083
max,199.873505,8880.000000,36.210526


In [11]:
# Data splitting
plantkeys = [
    'FBO0023', 'FSK0003', 'FSK0005', 'FSK0006', 'FSK0007', 'FSK0008', 'MIC0001',
    'SBO0001', 'SBO0002', 'SBO0005', 'SBO0007', 'SBO0009', 'SBO0010', 'SBO0011',
    'SBO0012', 'SBO0013', 'SBO0014', 'SBO0015', 'SBO0016', 'SBO0017', 'SBO0028',
    'SBO0029', 'SBO0030', 'SGA0001'
]



datasets = {}
count= 1

# Loop over each unique PlantKey in the DataFrame
for plantkey in data['PlantKey'].unique():
    
    # Filter data for the current PlantKey
    plant_data = data[data['PlantKey'] == plantkey]
    train_size = int(len(plant_data) * 0.6)  # Determining the size of the training set (70% of the data)
    train, test = plant_data.iloc[:train_size], plant_data.iloc[train_size:]  # Splitting the dataset into training and testing sets
    datasets[plantkey]= {'train': train, 'test': test}
    
#for plantkey in cleaned_data_final:






    # Retrieve the training and testing set for the current plantkey
    train_set = datasets[plantkey]['train']
    test_set = datasets[plantkey]['test']
    #train_set["ValueTime"] = pd.to_datetime(train_set["ValueTime"])
    #train_set.index.freq =train_set.index.freq = "D"
    #train_set.index.freq


    #test_set["ValueTime"] = pd.to_datetime(train_set["ValueTime"])
   # test_set.index.freq =train_set.index.freq = "D"
    #test_set.index.freq



# Convert 'ValueTime' to datetime and set frequency, if it's part of your dataset
   
for plantkey in plantkeys:
    
    # Save the datasets to CSV files
    train_set.to_csv(f'{plantkey}_train.csv')
    test_set.to_csv(f'{plantkey}_test.csv')

    print(f'Datasets for {plantkey} saved: {plantkey}_train.csv and {plantkey}_test.csv')
data.dtypes


Datasets for FBO0023 saved: FBO0023_train.csv and FBO0023_test.csv
Datasets for FSK0003 saved: FSK0003_train.csv and FSK0003_test.csv
Datasets for FSK0005 saved: FSK0005_train.csv and FSK0005_test.csv
Datasets for FSK0006 saved: FSK0006_train.csv and FSK0006_test.csv
Datasets for FSK0007 saved: FSK0007_train.csv and FSK0007_test.csv
Datasets for FSK0008 saved: FSK0008_train.csv and FSK0008_test.csv
Datasets for MIC0001 saved: MIC0001_train.csv and MIC0001_test.csv
Datasets for SBO0001 saved: SBO0001_train.csv and SBO0001_test.csv
Datasets for SBO0002 saved: SBO0002_train.csv and SBO0002_test.csv
Datasets for SBO0005 saved: SBO0005_train.csv and SBO0005_test.csv
Datasets for SBO0007 saved: SBO0007_train.csv and SBO0007_test.csv
Datasets for SBO0009 saved: SBO0009_train.csv and SBO0009_test.csv
Datasets for SBO0010 saved: SBO0010_train.csv and SBO0010_test.csv
Datasets for SBO0011 saved: SBO0011_train.csv and SBO0011_test.csv
Datasets for SBO0012 saved: SBO0012_train.csv and SBO0012_test

PlantKey        object
Ratio          float64
SunRad         float64
Temperature    float64
dtype: object

In [ ]:

for index, row in data.iterrows():
    train_set.to_csv(f'{plantkey}_train.csv')


In [ ]:
# For plantkey FBO0023
train_fbo0023 = pd.read_csv('FBO0023_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fbo0023 = pd.read_csv('FBO0023_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
print(train_fbo0023)

train_fbo0023.fillna(0, inplace=True)
test_fbo0023.fillna(0, inplace=True)


# For plantkey FSK0003
train_fsk0003 = pd.read_csv('FSK0003_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fsk0003 = pd.read_csv('FSK0003_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey FSK0005
train_fsk0005 = pd.read_csv('FSK0005_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fsk0005 = pd.read_csv('FSK0005_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey FSK0006
train_fsk0006 = pd.read_csv('FSK0006_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fsk0006 = pd.read_csv('FSK0006_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey FSK0007
train_fsk0007 = pd.read_csv('FSK0007_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fsk0007 = pd.read_csv('FSK0007_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey FSK0008
train_fsk0008 = pd.read_csv('FSK0008_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_fsk0008 = pd.read_csv('FSK0008_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey MIC0001
train_mic0001 = pd.read_csv('MIC0001_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_mic0001 = pd.read_csv('MIC0001_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0001
train_sbo0001 = pd.read_csv('SBO0001_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0001 = pd.read_csv('SBO0001_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')


# For plantkey SBO0002
train_sbo0002 = pd.read_csv('SBO0002_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0002 = pd.read_csv('SBO0002_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0005
train_sbo0005 = pd.read_csv('SBO0005_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0005 = pd.read_csv('SBO0005_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0007
train_sbo0007 = pd.read_csv('SBO0007_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0007 = pd.read_csv('SBO0007_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0009
train_sbo0009 = pd.read_csv('SBO0009_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0009 = pd.read_csv('SBO0009_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0010
train_sbo0010 = pd.read_csv('SBO0010_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0010 = pd.read_csv('SBO0010_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0011
train_sbo0011 = pd.read_csv('SBO0011_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0011 = pd.read_csv('SBO0011_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0012
train_sbo0012 = pd.read_csv('SBO0012_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0012 = pd.read_csv('SBO0012_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0013
train_sbo0013 = pd.read_csv('SBO0013_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0013 = pd.read_csv('SBO0013_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0014
train_sbo0014 = pd.read_csv('SBO0014_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0014 = pd.read_csv('SBO0014_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0015
train_sbo0015 = pd.read_csv('SBO0015_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0015 = pd.read_csv('SBO0015_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0016
train_sbo0016 = pd.read_csv('SBO0016_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0016 = pd.read_csv('SBO0016_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0017
train_sbo0017 = pd.read_csv('SBO0017_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0017 = pd.read_csv('SBO0017_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0028
train_sbo0028 = pd.read_csv('SBO0028_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0028 = pd.read_csv('SBO0028_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0029
train_sbo0029 = pd.read_csv('SBO0029_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0029 = pd.read_csv('SBO0029_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SBO0030
train_sbo0030 = pd.read_csv('SBO0030_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sbo0030 = pd.read_csv('SBO0030_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')

# For plantkey SGA0001
train_sga0001 = pd.read_csv('SGA0001_train.csv', index_col="ValueTime", parse_dates=True).asfreq('D')
test_sga0001 = pd.read_csv('SGA0001_test.csv', index_col="ValueTime", parse_dates=True).asfreq('D')



In [ ]:
def convertDate(Name):
    Name["ValueTime"] = pd.to_datetime(Name["ValueTime"])
    Name.index.freq =Name.index.freq = "D"
    Name.index.freq

arr = [train_fbo0023,test_fbo0023]
#for x in arr:
    #convertDate(x)




train_fbo0023.index.freq

In [ ]:
train_fbo0023.index.freq

train_fbo0023


In [ ]:
# Checking stationarity
adf_result = adfuller(train_fbo0023["Ratio"])  # Applying Augmented Dickey-Fuller test to check stationarity
print(adf_result)

In [ ]:
# Plotting the data
plt.figure(figsize=(6, 6))  # Setting the figure size for the plot
plt.plot(train_fbo0023["Ratio"], label='Train')  # Plotting the training data
plt.plot(test_fbo0023["Ratio"], label='Test')  # Plotting the testing data
plt.title('Ratio Over Time')  # Title of the plot
plt.xlabel('Date')  # X-axis label
plt.ylabel('Ratio')  # Y-axis label
plt.legend()  # Adding a legend to the plot
plt.show()  # Displaying the plot

In [ ]:
# ARIMA modeling
from pmdarima import auto_arima  # Importing the auto_arima function from pmdarima
  # Creating a new series 'train1' for ARIMA modeling
auto_model = auto_arima(train_fbo0023["Ratio"], seasonal=False, trace=True, error_action="ignore", suppress_warnings=True)  # Fitting the ARIMA model
auto_model.summary()  # Displaying the summary of the model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import matplotlib.pyplot as plt
# Further ARIMA modeling and forecasting
from statsmodels.tsa.arima.model import ARIMA  # Importing the ARIMA model
#data.index.freq = "1min"  # Setting the frequency of the data index to 1 minute
#data.index = pd.DatetimeIndex(data.index).to_period("1min")  # Converting the data index to DatetimeIndex with 1-minute period


In [ ]:
#np.asarray(train_fbo0023)

train_fbo0023.dtypes

In [ ]:




#train_fbo0023.index.freq =data.index.freq = "D"
train_fbo0023_ratio= train_fbo0023["Ratio"]

test_fbo0023_ratio= test_fbo0023["Ratio"]

model = ARIMA(train_fbo0023_ratio, order=(4, 0, 1))  # Initializing the ARIMA model with specified order
fitted_model = model.fit()  # Fitting the ARIMA model
fitted_model.summary()



In [ ]:
print(test_fbo0023_ratio.index.freq)

In [ ]:
forecast = fitted_model.forecast(steps=len(test_fbo0023_ratio))  # Forecasting using the fitted model
print(forecast)

In [ ]:
train_fbo0023_ratio.dtypes


In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
# Compare the forecasted values with the actual values
train_fbo0023_ratio.dtypes
#train_fbo0023_ratio.index = train_fbo0023_ratio.index.to_timestamp()
#test_fbo0023_ratio.index = test_fbo0023_ratio.index.to_timestamp()
#forecast.index = forecast.index.to_timestamp()
plt.figure(figsize=(10, 6))
plt.plot( train_fbo0023_ratio.index,train_fbo0023_ratio ,label='Train')
plt.plot( test_fbo0023_ratio.index,test_fbo0023_ratio ,label='Test')
plt.plot( test_fbo0023_ratio.index,forecast ,label='Forecast')
plt.title('Ratio Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Temperature Levels')
plt.legend()
plt.show()

#print(forecast.shape, test.shape)

# # Mean Absolute Error (MAE)
#MAE = np.mean(abs(forecast - test))
#print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))
# # Mean Absolute Error (MAE)
MAE = np.mean(abs(forecast - test_fbo0023_ratio))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

In [ ]:
train_fbo0023

test_fbo0023

In [ ]:
train_fbo0023_var = train_fbo0023.drop('PlantKey', axis=1)

test_fbo0023_var = test_fbo0023.drop('PlantKey', axis=1)

In [ ]:


test_fbo0023_var


test_fbo0023_var
# Assuming 'train_fbo0023_numeric' contains your data
# Calculate the first difference to make data stationary

train_fbo0023_var['SunRad_diff'] = train_fbo0023_var['SunRad'].diff()
train_fbo0023_var['Temperature_diff'] = train_fbo0023_var['Temperature'].diff()
test_fbo0023_var['SunRad_diff'] = test_fbo0023_var['SunRad'].diff()
test_fbo0023_var['Temperature_diff'] = test_fbo0023_var['Temperature'].diff()


train_fbo0023_var['SunRad'].diff()
train_fbo0023_var['Temperature'].diff()
test_fbo0023_var['SunRad'].diff()
test_fbo0023_var['Temperature'].diff()
#train_fbo0023_var['SunRad']= train_fbo0023_var['SunRad'].diff()
#train_fbo0023_var['Temperature']=train_fbo0023_var['Temperature'].diff()
# Drop the NaN values that come from differencing
train_fbo0023_var = train_fbo0023_var.dropna()
test_fbo0023_var= train_fbo0023_var.dropna()


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
gc_results = grangercausalitytests(train_fbo0023_var[['SunRad', 'Temperature']], maxlag=4)


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Function to perform the ADF test
def adf_test(series, title=''):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic for {title}: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis (Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

# Apply ADF test on the differenced data
adf_test(train_fbo0023_var['SunRad_diff'], 'SunRad_diff')
adf_test(train_fbo0023_var['Temperature_diff'], 'Temperature_diff')


In [ ]:
print(train_fbo0023_var)

In [ ]:
# Check for stationarity and difference if necessary
def check_stationarity(ts):
    result = adfuller(ts, autolag='AIC')
    if result[1] < 0.05:
        print('Series is stationary')
    else:
        print('Series is not stationary')

# Apply to all series
#for col in train_fbo0023_var.columns:
    #check_stationarity(train_fbo0023[col])

print(train_fbo0023_var)

#train_fbo0023.dtypes

In [ ]:
train_fbo0023_var.dtypes

#print(train_fbo0023_var['SunRad'].diff())



#print(train_fbo0023_var)

In [ ]:
train_fbo0023_var.dtypes


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR , VARMAX
from statsmodels.tsa.stattools import adfuller
from matplotlib import pyplot as plt

model = VARMAX(train_fbo0023_var, order=(4,0), enforce_stationarity= True)
fitted_model = model.fit(disp= False)
print(fitted_model.summary())



# The 'Ratio' forecast would be in forecast_df['Ratio']

In [ ]:
n_forecast = len(test_fbo0023_var)  # Ensure this matches the test set size if comparing directly
start_forecast = len(train_fbo0023_var)
end_forecast = start_forecast + n_forecast - 1

predictions = fitted_model.get_prediction(start=start_forecast, end=end_forecast)
forecast_mean = predictions.predicted_mean
forecast_mean

In [ ]:
forecast_mean.columns= ["Ratio", "SunRad", "temp"]
forecast_mean

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
# Compare the forecasted values with the actual values
train_fbo0023_ratio.dtypes
#train_fbo0023_ratio.index = train_fbo0023_ratio.index.to_timestamp()
#test_fbo0023_ratio.index = test_fbo0023_ratio.index.to_timestamp()
#forecast.index = forecast.index.to_timestamp()
plt.figure(figsize=(10, 6))
plt.plot( train_fbo0023_var.index,train_fbo0023_var["Ratio"] ,label='Train')
plt.plot( test_fbo0023_var.index,test_fbo0023_var["Ratio"] ,label='Test')
plt.plot( test_fbo0023_var.index,forecast_mean["Ratio"] ,label='Forecast')
plt.title('Ratio Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Temperature Levels')
plt.legend()
plt.show()

#print(forecast.shape, test.shape)

# # Mean Absolute Error (MAE)
#MAE = np.mean(abs(forecast - test))
#print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))
# # Mean Absolute Error (MAE)

MAE = np.mean(abs(forecast - test_fbo0023_ratio))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

In [ ]:
# Calculate daily absolute errors
daily_errors = np.abs(forecast_mean['Ratio'] - test_fbo0023_var['Ratio'])

# Optionally, view the daily errors
print(daily_errors)
